
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 2.1 DEMO: Implementing Delta Sharing (Databricks-to-Databricks)

## Overview
In this demo, we will implement Delta Sharing to securely share data between Databricks workspaces. This demonstration is split into two parts:

**Provider Notebook (This Notebook):** We will create a Unity Catalog with sample customer and sales data, configure Delta Sharing by creating a share, set up a recipient organization, and generate an activation link for secure access.

**Recipient Notebook:** The receiving organization (using Databricks) will mount the share in a local catalog and query the shared tables directly without copying data, and perform analytics using the shared datasets.

This demo showcases the Databricks-to-Databricks (D2D) sharing pattern, where both the provider and recipient use Databricks workspaces. You'll see how Delta Sharing enables secure, live data sharing without data duplication or complex ETL processes.

### Learning Objectives
By the end of this demo, you will be able to:
1. Create and configure Unity Catalog resources (catalogs, schemas, and tables) suitable for Delta Sharing.
2. Implement Delta Sharing by creating a share and adding tables to it for external access.
3. Create and configure recipients to control who can access your shared data.
4. Understand the provider's role in the Delta Sharing architecture and how to manage shared data access.
5. Verify and monitor shared data configurations using SQL commands in the provider workspace.

## Background

**Scenario:**
You are a data provider company (*"Acme Corp"*) that wants to share customer analytics data with partner organizations using Delta Sharing. In this notebook, you'll set up the provider side of Delta Sharing by creating the data, configuring shares, and generating access credentials for your recipients.

In [0]:
%run ./Includes/Demo-Setup-2

## Inspect catalog, schema and tables

The setup script has set up a catalog, schema and tables for our fictitious company which will act as the __provider__, lets examine these now:

In [0]:
-- get current context
SELECT current_catalog(), current_schema()

In [0]:
-- show tables in our schema
SHOW TABLES;

In [0]:
-- show the data in the customers table
SELECT * FROM customers;

In [0]:
-- show the data in the sales_transactions table
SELECT * FROM sales_transactions;

## Step 1: Create a Delta Share

Now we'll create a share and add our tables to it. A share is a logical grouping of tables that can be shared with recipients.

In [0]:
-- Create a new share
CREATE SHARE IF NOT EXISTS internal_retail
COMMENT 'Share containing customer and sales data for partner analysis';

-- View all shares
SHOW SHARES;

## Step 2: Add Assets to the Share

Add the `customers` and `sales_transactions` tables to the share. Once added, these tables become available for recipients to query through the Delta Sharing protocol.

In [0]:
-- Add tables to the share
ALTER SHARE internal_retail 
ADD TABLE customers
COMMENT 'Customer master data';

ALTER SHARE internal_retail 
ADD TABLE sales_transactions
COMMENT 'Sales transaction history';

-- View what's in the share
SHOW ALL IN SHARE internal_retail

## Step 3: Create a Recipient (Databricks-to-Databricks)

Create a recipient for Databricks-to-Databricks sharing. For D2D sharing, the recipient needs to provide their metastore ID .

In [0]:
-- Create a Databricks-to-Databricks recipient
CREATE RECIPIENT IF NOT EXISTS west_division
COMMENT 'Recipient for D2D Delta Sharing demo';

-- View all recipients
SHOW RECIPIENTS;

In [0]:
-- Grant the share to the recipient
GRANT SELECT ON SHARE ${c.share_name} TO RECIPIENT ${c.d2d_recipient};

-- Describe the recipient to get the activation link
DESCRIBE RECIPIENT ${c.d2d_recipient};

## Step 5: Get Activation Link

**Important:** Copy the `activation_link` from the output above and share it with the recipient. The recipient will use this link to activate the share in their workspace.

The activation link will look like:
```
https://accounts.cloud.databricks.com/...
```

## Step 6: Monitor and Manage the Share

Let's review what we've created and explore monitoring capabilities.

In [0]:
-- View share details
DESCRIBE SHARE ${c.share_name};

-- List all tables in the share
SHOW ALL IN SHARE ${c.share_name};

-- View recipient details
DESCRIBE RECIPIENT ${c.d2d_recipient};

## Summary

✅ **What we accomplished:**

1. Created a Unity Catalog and schema for our data
2. Created sample customers and sales_transactions tables
3. Created a Delta Share and added tables to it
4. Created a Databricks-to-Databricks recipient
5. Granted the share to the recipient
6. Generated an activation link for the recipient

**Next Steps:**
- Share the activation link with the recipient workspace
- The recipient will use the activation link to create a catalog in their workspace
- The recipient can then query the shared data

**Key Concepts:**
- **Share**: A logical grouping of tables that can be shared
- **Recipient**: An entity that receives access to a share
- **Activation Link**: A one-time link used to activate the share in the recipient workspace
- **D2D Sharing**: Direct sharing between two Databricks workspaces using Unity Catalog

&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>